<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/4.%20%E7%94%BB%E5%83%8F%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google検索から画像をスクレイピングする

Chromedriverをインストール

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium==4.1.0 #新しいバージョンだとエラーが出るので旧バージョンにする

google_images_dowloadをインストール

In [ ]:
# #今はエラーになるので下記のパッチを用いる
# #!pip install google_images_download

# !pip uninstall google_images_download
# !git clone https://github.com/Joeclinton1/google-images-download.git
# %cd google-images-download
# !python setup.py install

# **スクレイピングする**

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Goodle drive内にダウンロードする場合

In [ ]:
# from google_images_download import google_images_download
# response = google_images_download.googleimagesdownload()

# response.download({
#     "keywords": "イチゴ",
#     "limit": 3,
#     "no_numbering": True,
#     "output_directory": "/content/drive/My Drive/AI_laboratory_course/images",
#     "image_directory": "strawberry",
#     "format": "jpg" or "jpeg",
#     "print_urls": False,
#     "chromedriver": "/usr/lib/chromium-browser/chromedriver"  #chromedriverをダウンロードして指定の場所に置いておく
#     })


# response.download({
#     "keywords": "リンゴ",
#     "limit": 3,
#     "no_numbering": True,
#     "output_directory": "/content/drive/My Drive/AI_laboratory_course/images",
#     "image_directory": "apple",
#     "print_urls": False,
#     "format": "jpg" or "jpeg",
#      "chromedriver": "/usr/lib/chromium-browser/chromedriver"
#     })

## Contentフォルダにダウンロードする場合

In [ ]:
# from google_images_download import google_images_download
# response = google_images_download.googleimagesdownload()

# response.download({
#     "keywords": "イチゴ",
#     "limit": 3,
#     "no_numbering": True,
#     "output_directory": "/content",
#     "image_directory": "strawberry",
#     "format": "jpg" or "jpeg",
#     "print_urls": False,
#     "chromedriver": "/usr/lib/chromium-browser/chromedriver"  #chromedriverをダウンロードして指定の場所に置いておく
#     })


# response.download({
#     "keywords": "リンゴ",
#     "limit": 3,
#     "no_numbering": True,
#     "output_directory": "/content",
#     "image_directory": "apple",
#     "print_urls": False,
#     "format": "jpg" or "jpeg",
#      "chromedriver": "/usr/lib/chromium-browser/chromedriver"
#     })

In [25]:
QUERY = 'パイナップル　フリー'                        # 検索ワード
SAVE_DIR = '/content/pine'                        # 出力フォルダへのパス（フォルダがない場合は自動生成する）
FILE_NAME = 'pine_'                          # ファイル名（ファイル名の後ろに０からの連番と拡張子が付く）


import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
 
tm_start = time.time()            #処理時間計測用
dt_now = datetime.datetime.now()  # 現在日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M')
print(dt_date_str)
 
# QUERY = 'りんご　フリー'                        # 検索ワード
LIMIT_DL_NUM = 10                          # ダウンロード数の上限
# SAVE_DIR = '/content/apple'                        # 出力フォルダへのパス（フォルダがない場合は自動生成する）
# FILE_NAME = 'strawberry_'                          # ファイル名（ファイル名の後ろに０からの連番と拡張子が付く）
TIMEOUT = 60                                # 要素検索のタイムアウト（秒）
ACCESS_WAIT = 1                             # アクセスする間隔（秒）
RETRY_NUM = 3                               # リトライ回数（クリック、requests）
DRIVER_PATH = '/usr/lib/chromium-browser/chromedriver'       # chromedriver.exeへのパス
 
# Chromeをヘッドレスモードで起動
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
driver = webdriver.Chrome(DRIVER_PATH, options=options)
 
# タイムアウト設定
driver.implicitly_wait(TIMEOUT)
 
tm_driver = time.time()
print('WebDriver起動完了', f'{tm_driver - tm_start:.1f}s')
 
# Google画像検索ページを取得
url = f'https://www.google.com/search?q={QUERY}&tbm=isch'
driver.get(url)
 
tm_geturl = time.time()
print('Google画像検索ページ取得', f'{tm_geturl - tm_driver:.1f}s')
 
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
 
count = len(tmb_alts) - tmb_alts.count('')
print(count)
 
while count < LIMIT_DL_NUM:
    # ページの一番下へスクロールして新しいサムネイル画像を表示させる
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
 
    # サムネイル画像取得
    tmb_elems = driver.find_elements_by_css_selector('#islmp img')
    tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
 
    count = len(tmb_alts) - tmb_alts.count('')
    print(count)  
 
# サムネイル画像をクリックすると表示される領域を取得
imgframe_elem = driver.find_element_by_id('islsp')
 
# 出力フォルダ作成
os.makedirs(SAVE_DIR, exist_ok=True)
 
# HTTPヘッダ作成
HTTP_HEADERS = {'User-Agent': driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)           
            
# ダウンロード対象のファイル拡張子
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.gif')
 
# 拡張子を取得
def get_extension(url):
    url_lower = url.lower()
    for img_ext in IMG_EXTS:
        if img_ext in url_lower:
            extension = '.jpg' if img_ext == '.jpeg' else img_ext
            break
    else:
        extension = ''
    return extension
 
# urlの画像を取得しファイルへ書き込む
def download_image(url, path, loop):
    result = False
    for i in range(loop):
        try:
            r = requests.get(url, headers=HTTP_HEADERS, stream=True, timeout=10)
            r.raise_for_status()
            with open(path, 'wb') as f:
                f.write(r.content)

        except requests.exceptions.SSLError:
            print('***** SSL エラー')
            break  # リトライしない
        except requests.exceptions.RequestException as e:
            print(f'***** requests エラー({e}): {i + 1}/{RETRY_NUM}')
            time.sleep(1)
        else:
            result = True
            break  # try成功
    return result
 
tm_thumbnails = time.time()
print('サムネイル画像取得', f'{tm_thumbnails - tm_geturl:.1f}s')
 
# ダウンロード
EXCLUSION_URL = 'https://lh3.googleusercontent.com/'  # 除外対象url
count = 0
url_list = []
for tmb_elem, tmb_alt in zip(tmb_elems, tmb_alts):
     
    if tmb_alt == '':
        continue
 
    print(f'{count}: {tmb_alt}')
 
    for i in range(RETRY_NUM):
        try:
            # サムネイル画像をクリック
            tmb_elem.click()
        except ElementClickInterceptedException:
            print(f'***** click エラー: {i + 1}/{RETRY_NUM}')
            driver.execute_script('arguments[0].scrollIntoView(true);', tmb_elem)
            time.sleep(1)
        else:
            break  # try成功
    else:
        print('***** キャンセル')
        continue  # リトライ失敗
         
    # アクセス負荷軽減用のウェイト
    time.sleep(ACCESS_WAIT)
     
    alt = tmb_alt.replace("'", "\\'")
    try:
        img_elem = imgframe_elem.find_element_by_css_selector(f'img[alt=\'{alt}\']')
    except NoSuchElementException:
        print('***** img要素検索エラー')
        print('***** キャンセル')
        continue
 
    # url取得
    tmb_url = tmb_elem.get_attribute('src')  # サムネイル画像のsrc属性値
 
    for i in range(RETRY_NUM):
        url = img_elem.get_attribute('src')
        if EXCLUSION_URL in url:
            print('***** 除外対象url')
            url = ''
            break
        elif url == tmb_url:  # src属性値が遷移するまでリトライ
            print(f'***** urlチェック: {i + 1}/{RETRY_NUM}')
            time.sleep(1)
            url = ''
        else:
            break
 
    if url == '':
        print('***** キャンセル')
        continue
 
    # 画像を取得しファイルへ保存
    ext = get_extension(url)
    if ext == '':
        print(f'***** urlに拡張子が含まれていないのでキャンセル')
        print(f'{url}')
        continue
 
    filename = f'{FILE_NAME}{count}{ext}'
    path = SAVE_DIR + '/' + filename
    result = download_image(url, path, RETRY_NUM)
    if result == False:
        print('***** キャンセル')
        continue
    url_list.append(f'{filename}: {url}')
 
    # ダウンロード数の更新と終了判定
    count += 1
    if count >= LIMIT_DL_NUM:
        break
 
tm_end = time.time()
print('ダウンロード', f'{tm_end - tm_thumbnails:.1f}s')
print('------------------------------------')
total = tm_end - tm_start
total_str = f'トータル時間: {total:.1f}s({total/60:.2f}min)'
count_str = f'ダウンロード数: {count}'
print(total_str)
print(count_str)
 
# # urlをファイルへ保存
# path = SAVE_DIR + '/url/' + '_url.txt'
# with open(path, 'w', encoding='utf-8') as f:
#     f.write(dt_date_str + '\n')
#     f.write(total_str + '\n')
#     f.write(count_str + '\n')
#     f.write('\n'.join(url_list))
 
driver.quit()

2023/01/17 03:12


<ipython-input-25-3b1166cee4cc>:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH, options=options)


WebDriver起動完了 1.0s
Google画像検索ページ取得 2.7s


<ipython-input-25-3b1166cee4cc>:52: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')


48
{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/108.0.5359.71 Safari/537.36'}
サムネイル画像取得 1.3s
0: パイナップル 写真素材 [ 2642245 ] 無料 - フォトライブラリー photolibrary


<ipython-input-25-3b1166cee4cc>:71: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  imgframe_elem = driver.find_element_by_id('islsp')
/usr/local/lib/python3.8/dist-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


***** urlチェック: 1/3
1: パイナップル02 | フリー素材ドットコム
2: パイナップルのイラスト | 商用OKの無料イラスト素材サイト ツカッテ
3: パイナップル フリー素材のイラスト・画像集めてみた！
4: シンプルなパイナップル 写真素材 [ 2635235 ] 無料 - フォトライブラリー photolibrary
5: パイナップルAの無料素材 - イラスト沖縄（おきなわ）
6: パイナップル | フリー素材ドットコム
7: パイナップル（パイン）のイラスト素材 | 商用可能な無料(フリー)のイラスト素材ならストックマテリアル
8: フリー写真] カットしたパイナップル - パブリックドメインQ：著作権フリー画像素材集
***** requests エラー(403 Client Error: Forbidden for url: https://sun.publicdomainq.net/202002/08o/publicdomainq-0042631suxbus.jpg): 1/3
***** requests エラー(403 Client Error: Forbidden for url: https://sun.publicdomainq.net/202002/08o/publicdomainq-0042631suxbus.jpg): 2/3
***** requests エラー(403 Client Error: Forbidden for url: https://sun.publicdomainq.net/202002/08o/publicdomainq-0042631suxbus.jpg): 3/3
***** キャンセル
8: パイナップル 白背景の写真素材｜写真素材なら「写真AC」無料（フリー）ダウンロードOK
9: かわいいパイナップルのキャラクターのイラスト | 無料のフリー素材 イラストエイト
ダウンロード 27.0s
------------------------------------
トータル時間: 32.1s(0.53min)
ダウンロード数: 10


#宿題：

次回、実際にdeep learningを用いた画像の分類を行います。
1. 2つのカテゴリーの画像をダウンロードしましょう
2. 実際に1枚ずつ確認し、不適切なものを除きましょう
